<a href="https://colab.research.google.com/github/singularity014/T5Transformer_text_gen/blob/main/TextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install transformers==2.9.0

     |████████████████████████████████| 645kB 20.7MB/s 
     |████████████████████████████████| 1.2MB 55.6MB/s 
     |████████████████████████████████| 890kB 55.6MB/s 
     |████████████████████████████████| 5.6MB 17.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f274519188f1f4bfdff0f449b4a558998efae731a2fdbce7b12d82773e8841f1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [9]:
# Check we have a GPU and check the memory size of the GUP
!nvidia-smi

Tue Mar 16 23:24:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:

import random
import pandas as pd
import numpy as np        
import torch
from torch.utils.data import Dataset, DataLoader

In [4]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [5]:
tokenizer = T5Tokenizer.from_pretrained('t5-large')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-large')


In [6]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

In [7]:
# dataset preparation

true_false_adjective_tuples = [
                               ("We help cross-border e-commerce sell more by providing the local services shoppers need.",
                                "So, you are all ready to conquer the world? Don't forget to first conquer your local area. You can do this by providing the customer service they expect and deserve."),
                               
                               ("We help cross-border e-commerce have more customers by providing the local services shoppers need.","We make your store cross-border ready"),
                               ("We help cross-border e-commerce get more sale by providing the local services shoppers need.","Protect your business, sell more online, nationally and internationally."),
                               ("What makes us exceptional? We combine export and digital skills to deliver cross-border ecommerce solutions that have a multichannel approach",
                                "With coronavirus spreading everywhere, online shopping is now an absolute must for every company out there! To be successful in the ever-competitive landscape, businesses need to mix cross-border commerce and e-commerce to grow their online sales. That’s the only way to maintain business and keep pace."),
                            #    ("","The bathroom is dirty"),
                            #    ("The exam was easy","The exam was difficult"),
                            #    ("The house is big","The house is small"),
                            #    ("The house owner is good","The house owner is bad"),
                            #    ("The little kid is fat","The little kid is thin"),
                            #    ("She arrived early","She arrived late."),
                            #    ("John is very hardworking","John is very lazy"),
                            #    ("The fridge is empty","The fridge is full")

]

In [ ]:
t5_model.train()

epochs = 10

for epoch in range(epochs):
  print ("epoch ",epoch)
  for input,output in true_false_adjective_tuples:
    input_sent = "falsify: "+input+ " </s>"
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]

    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]


    # the forward function automatically creates the correct decoder_input_ids
    output = t5_model(input_ids=input_ids, lm_labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()




epoch  0


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
